In [16]:
import os
import pandas as pd
from rdkit import Chem
from pathlib import Path 


## Benchmark meta dataset

CDK2 highly flexible kinase and a major cancer target.
Related dataset with the ensemble of five representative conformations in the ensemble.


Ensemble:
- 2R64 CDK2 in complex with an inhibitor (A-loop flexible, no cyclin)
- 2R3K CDK2 in complex with an inhibitor (A-loop flexible, no cyclin)
- 3BHV CDK2 in complex with an inhibitor (A-loop out, with cyclin)
- 3BHU CDK2 in complex with an inhibitor (A-loop out, with cyclin)
- 8FP5 CDK2 in complex with ATP (A-loop in, no cyclin)

In [17]:
receptor_categories = {
    "8fp5_receptor_aligned": 0,
    "2r64_receptor_aligned": 0,
    "2r3k_receptor_aligned": 0,
    "3bhu_receptor_aligned": 1,
    "3bhv_receptor_aligned": 1,
}

ligand_categories = {
    "A-loop in, with cyclin": 0,
    "A-loop in, no cyclin": 0,
    "A-loop in, no cyclin": 0,
    "A-loop flex, no cyclin": 1,
    "A-loop out, with cyclin": 1
}

In [18]:
full_dataset = pd.read_csv("./CDK2/CDK2_selected_ligands.csv")

In [19]:
full_dataset["ligand_category"] = full_dataset["comment"].apply(lambda x: ligand_categories[x])

In [20]:
full_dataset = full_dataset[["pdb_id", "ligands", "rot_bonds", "n_atoms", "comment", "ligand_category"]]
full_dataset

,pdb_id,ligands,rot_bonds,n_atoms,comment,ligand_category
0,2G9X,NU5,12,34,"A-loop in, with cyclin",0
1,3ROY,22Z,11,27,"A-loop in, no cyclin",0
2,3R71,X86,10,26,"A-loop in, no cyclin",0
3,1H07,MFQ,10,33,"A-loop in, no cyclin",0
4,6RIJ,K4W,10,27,"A-loop in, with cyclin",0
5,2IW6,QQ2,10,33,"A-loop out, with cyclin",1
6,6INL,AJR,10,29,"A-loop flex, no cyclin",1
7,3RAI,X85,10,30,"A-loop in, no cyclin",0
8,3RPO,24Z,10,27,"A-loop in, no cyclin",0
9,1H07,MFP,10,33,"A-loop in, no cyclin",0


In [21]:
import glob 
# find the ligands for which we have results
ens_res_list = []
resuls_pd = None
for i, row in full_dataset.iterrows():
    pdb_id = row["pdb_id"]
    pdb_id_l = pdb_id.lower()
    data_dir = Path("./CDK2/ligands/{}".format(pdb_id_l))
    if data_dir.exists():
        ligands_in_dir = glob.glob(str(data_dir / "{}_ligand*.mol2".format(pdb_id_l)))
        for ligand in ligands_in_dir:
            lig_name = Path(ligand).stem
            output_dir = "../../benchmark-ens-results/{}/".format(lig_name)
            results_file = Path(output_dir) / "analysis/results.csv"
            if results_file.exists():
                ens_res_list.append(lig_name) 
                if resuls_pd is None:
                    resuls_pd = pd.read_csv(results_file)
                    resuls_pd["ligand_name"] = [lig_name for i in range(resuls_pd.shape[0])]
                else:
                    tmp = pd.read_csv(results_file)
                    tmp["ligand_name"] = [lig_name for i in range(tmp.shape[0])]
                    resuls_pd = pd.concat([resuls_pd, tmp])
print(len(ens_res_list))


21


In [22]:
resuls_pd[resuls_pd["ligand_name"]=="6inl_ligand"].sort_values(by="energies").sort_values(by="rmsds")

,Unnamed: 0.2,Unnamed: 0.1,energies,rmsds,model_id,replica_id,receptor_id,fragment_id,thread_id,Unnamed: 0,job_id,ligand_name
46,46,2,-8.984,1.571993,0.0,3.0,8fp5_receptor_aligned,2.0,3.0,0.0,job_8fp5_receptor_aligned_6inl_ligand,6inl_ligand
45,45,2,-8.984,1.571993,0.0,3.0,8fp5_receptor_aligned,2.0,3.0,0.0,job_8fp5_receptor_aligned_6inl_ligand,6inl_ligand
44,44,2,-8.984,1.571993,0.0,3.0,8fp5_receptor_aligned,2.0,3.0,0.0,job_8fp5_receptor_aligned_6inl_ligand,6inl_ligand
47,47,2,-8.984,1.571993,0.0,3.0,8fp5_receptor_aligned,2.0,3.0,0.0,job_8fp5_receptor_aligned_6inl_ligand,6inl_ligand
63,63,3,-8.979,1.575625,0.0,2.0,8fp5_receptor_aligned,2.0,2.0,0.0,job_8fp5_receptor_aligned_6inl_ligand,6inl_ligand
...,...,...,...,...,...,...,...,...,...,...,...,...
481,481,22,-6.684,10.796423,6.0,2.0,3bhv_receptor_aligned,2.0,2.0,6.0,job_3bhv_receptor_aligned_6inl_ligand,6inl_ligand
417,417,22,-6.800,10.835908,6.0,3.0,2r3k_receptor_aligned,2.0,3.0,6.0,job_2r3k_receptor_aligned_6inl_ligand,6inl_ligand
418,418,22,-6.800,10.835908,6.0,3.0,2r3k_receptor_aligned,2.0,3.0,6.0,job_2r3k_receptor_aligned_6inl_ligand,6inl_ligand
419,419,22,-6.800,10.835908,6.0,3.0,2r3k_receptor_aligned,2.0,3.0,6.0,job_2r3k_receptor_aligned_6inl_ligand,6inl_ligand


In [23]:
resuls_pd["ligand_categories"] = resuls_pd["ligand_name"].apply(lambda x: full_dataset[full_dataset["pdb_id"]==x.split("_")[0].upper()].iloc[0]["ligand_category"])

In [24]:
resuls_pd["receptor_categories"] = resuls_pd["receptor_id"].apply(lambda x: receptor_categories[x])

In [51]:
results_summary = resuls_pd.sort_values("energies", ascending=True).groupby("ligand_name").head(10).groupby("ligand_name")["rmsds"].min().reset_index()
results_summary.columns = ["ligand_name", "best_rmsds_top5"]
results_summary ["best_rmsds"] = resuls_pd.sort_values("energies", ascending=True).groupby("ligand_name")["rmsds"].min().reset_index()["rmsds"]
#results_summary ["selected_rec"] = resuls_pd.sort_values("energies", ascending=True).groupby("ligand_name")["rmsds"].min().reset_index()["receptor_id"]
results_summary ["best_rmsds_top1"] = resuls_pd.sort_values("energies", ascending=True).groupby("ligand_name").head(1).groupby("ligand_name")["rmsds"].min().reset_index()["rmsds"]
results_summary["mean_rmsds_top5"] = resuls_pd.sort_values("energies", ascending=True).groupby("ligand_name").head(5).groupby("ligand_name")["rmsds"].mean().reset_index()["rmsds"]
results_summary["std_rmsds_top5"] = resuls_pd.sort_values("energies", ascending=True).groupby("ligand_name").head(5).groupby("ligand_name")["rmsds"].std().reset_index()["rmsds"]

In [52]:
results_summary = results_summary.sort_values("best_rmsds").reset_index(drop=True)
results_summary

,ligand_name,best_rmsds_top5,best_rmsds,best_rmsds_top1,mean_rmsds_top5,std_rmsds_top5
0,3ns9_ligand,6.888610,1.349567,6.895128,6.893824,0.002915
1,3r71_ligand,1.844475,1.467957,5.904052,5.899827,0.009446
2,6inl_ligand,1.587441,1.571993,1.587556,1.587533,0.000051
3,3rai_ligand,9.939425,1.606133,9.946488,9.948854,0.005290
4,3r73_ligand,5.750467,1.711741,5.754142,5.754216,0.000166
5,3ddp_ligand,4.492268,2.089100,4.502001,4.500054,0.004353
6,1h00_ligand_fcp,3.524417,3.005377,3.524417,3.524417,0.000000
7,1h00_ligand_fap,4.667915,3.262525,4.667915,4.667915,0.000000
8,3eid_ligand,5.173122,3.668447,5.173122,5.176745,0.008102
9,1h08_ligand,5.590708,3.734912,5.623077,5.618975,0.009173


In [27]:
selected_ligands = list(results_summary[:6]["ligand_name"])
selected_ligands

['3ns9_ligand',
 '3r71_ligand',
 '6inl_ligand',
 '3rai_ligand',
 '3r73_ligand',
 '3ddp_ligand']

In [62]:
import plotly.express as px
ligand_titles = {0: "A-loop IN", 1: "A-loop OUT"}
receptor_titles = {0: "A-loop IN", 1: "A-loop OUT"}
tmp_res = resuls_pd.sort_values("energies").groupby("ligand_name").head(10).reset_index()
tmp_res = tmp_res.sort_values("rmsds").groupby("ligand_name").head(1).reset_index()
tmp_res = tmp_res[["energies", "rmsds","receptor_categories", "ligand_categories"]]
tmp_res["receptor_categories"] = tmp_res["receptor_categories"].apply(lambda x: receptor_titles[x])
tmp_res["ligand_categories"] = tmp_res["ligand_categories"].apply(lambda x:ligand_titles[x])
tmp_res.columns = ["Best in Top 10 Energy (kcal/mol)", "Best in Top 10 RMSD", "Receptor Conformation Type", "Ligand Type"]

fig = px.box(tmp_res.sort_values("Receptor Conformation Type"), 
             x='Ligand Type', y='Best in Top 10 RMSD', 
             color='Receptor Conformation Type', points="all", 
             color_discrete_sequence = ["#FF6C90", "#00B0F6"],
             template="ggplot2")
'''
fig = px.scatter(tmp_res.sort_values("Receptor Conformation Type"), 
             x='Ligand Type', y='Best in Top 10 RMSD', 
             color='Receptor Conformation Type',
             color_discrete_sequence = ["#FF6C90", "#00B0F6"],
             template="ggplot2")
'''
fig.update_traces(marker_size=10)
fig.show()
fig.write_image("CDK2_results.svg")

In [63]:
fig._data_objs[0].pointpos = 0

fig._data_objs[0].jitter = 1
fig._data_objs[0].line = dict(color = 'rgba(0,0,0,0)')
fig._data_objs[0].fillcolor = 'rgba(0,0,0,0)'
fig._data_objs[1].pointpos = 0
fig._data_objs[1].line = dict(color = 'rgba(0,0,0,0)')
fig._data_objs[1].fillcolor = 'rgba(0,0,0,0)'


In [64]:
fig.show()

In [1]:
import plotly.io as pio
templ = pio.templates["ggplot2"]

In [13]:
templ._orphan_props["layout"]["colorscale"]["sequential"]

[[0, 'rgb(20,44,66)'], [1, 'rgb(90,179,244)']]